In [ ]:
# Args
in_folder = "/content/SRN/"
out_folder = "/content/processing/"

hf_repo_name = "MVP-Group-Project/srn_cars_priors"

git_token = ""
hf_token = ""

In [ ]:
!cd /content/
!git clone https://github.com/Kacper-M-Michalik/3dgs-priors.git

In [ ]:
import os
from pathlib import Path
import glob
from huggingface_hub import login, HfApi
from datasets import load_dataset, load_from_disk

In [ ]:
out_folder_path = Path(out_folder)
out_folder_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Create venv for each prior model
!python3 -m venv /content/models/depth --without-pip
!python3 -m venv /content/models/normal --without-pip

# Have to manually install pip to correctly build venvs on colab
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/models/depth/bin/python3 get-pip.py
!/content/models/normal/bin/python3 get-pip.py

# Verify venv's work
!/content/models/depth/bin/pip --version
!/content/models/normal/bin/pip --version

!ls -l /content/models/

In [ ]:
# Setup models

# Depth
!/content/models/depth/bin/pip install -r /content/3dgs-priors/geometry-priors/depth_requirements.txt

# Normals
!git clone https://github.com/baegwangbin/surface_normal_uncertainty.git /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/
!mkdir /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/checkpoints/
!gdown 1lOgY9sbMRW73qNdJze9bPkM2cmfA8Re- -O /content/3dgs-priors/geometry-priors/surface_normal_uncertainty/checkpoints/scannet.pt
!/content/models/normal/bin/pip install -r /content/3dgs-priors/geometry-priors/normal_requirements.txt

In [ ]:
# Get SRN cars dataset
%cd /content
!mkdir SRN
%cd /content/SRN
!mkdir srn_cars
%cd /content/SRN/srn_cars
!gdown --id 19yDsEJjx9zNpOKz9o6AaK-E8ED6taJWU -O cars.zip
!unzip cars.zip
%cd /content

In [ ]:
# Launch models to process dataset
!/content/models/normal/bin/python /content/3dgs-priors/geometry-priors/generate_normal.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=250    

In [ ]:
!/content/models/depth/bin/python /content/3dgs-priors/geometry-priors/generate_depth.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=250

In [ ]:
!python /content/3dgs-priors/geometry-priors/generate_base.py --in_folder="{in_folder}" --out_folder="{out_folder}" --save_iter=500

In [ ]:
# Split generated parquets into correct folder/file structure for HuggingFace upload
%cd /content/
!mkdir upload
%cd upload

datatypes = ["rgbs", "intrins", "poses", "depths", "normals"]
splits = ["train", "test", "val"]

for datatype in datatypes:
    # Make a directory for the current dataset file
    !mkdir {datatype}
    %cd {datatype}
    load_path = os.path.join(out_folder, "srn_cars_" + datatype + ".parquet")
    dataset = load_dataset("parquet", data_files=load_path)['train']
    for split in splits:    
        # Generate split specific directory
        !mkdir {split} 
        save_path = "/content/upload/{}/{}/{}.parquet".format(datatype, split, datatype) 
        filter_split = lambda data: data['split'] == split
        filtered = dataset.filter(filter_split).to_pandas().drop(columns=['split'])
        filtered.to_parquet(save_path)
    %cd ..

In [ ]:
# Upload dataset folder to HuggingFace
login(token=hf_token)

api = HfApi()
api.upload_folder(
    folder_path="/content/upload",
    repo_id="MVP-Group-Project/srn_cars_priors",
    repo_type="dataset",
    path_in_repo="." 
)